In [15]:
import numpy as np 
import time
import pandas as pd
import json 
import requests 
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim 
import folium 
import folium 
from folium import plugins
import matplotlib.cm as cm
import matplotlib.colors as colors
import seaborn as sns
from sklearn.cluster import KMeans

**Downloading the neighborhoods of New York**

In [28]:
#In case you want to download the dataset you can uncomment the below line to download it
#!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
with open(r'C:\Users\Jesus\Downloads\nyu_2451_34572-geojson.json') as json_data:
    newyork_data = json.load(json_data)

In [33]:
#neighborhoods_data = newyork_data['features']
#neighborhoods_data[0]
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 
# Create the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
#Filling the data into the dataframe
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


**Selecting the city of interest:**

In [35]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


**Creating the map with the neighborhoods on top**

In [36]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

**As we are focussing on Manhattan we can filter the original Borough dataframe**

In [41]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


**We locate the coordinates of Manhattan to reduce the search**

In [39]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [42]:
# create map of Manhattan using latitude and longitude values
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

**We use Foursquare to query the Venues in Manhattan**

In [49]:
CLIENT_ID = 'UDHWCQOFGS5VFVQUXZWNLEQHJUEOSGGG1ND1AIXFIHRQ35L2' # your Foursquare ID
CLIENT_SECRET = '45EH01S4XD2J5RJSCRW4T1QRUROH1GSMYH3WN0ZL4ZZBUVFB' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#This function will scann 500meters arround the neighborhoods and collect all the venues:
def getNearbyVenues(names, latitudes, longitudes, radius=500):    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#Execute the function for all the neighborhoods of manhattan
manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [45]:
print(manhattan_venues.shape)
manhattan_venues.head()

(3279, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.91066,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.876551,-73.91066,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.876551,-73.91066,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.876551,-73.91066,Dunkin',40.877136,-73.906666,Donut Shop
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop


**We filter the Venues based on the likes of the customer**

In [55]:
#Customer venues requirements
filtered_manhattan_venues=manhattan_venues[manhattan_venues['Venue Category'].str.contains('Metro Station|Coffee Shop|Restaurant|Wine|Gym|Food',na=False,case=False)]
filtered_manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
4,Marble Hill,40.876551,-73.91066,Starbucks,40.877531,-73.905582,Coffee Shop
5,Marble Hill,40.876551,-73.91066,Astral Fitness & Wellness Center,40.876705,-73.906372,Gym
6,Marble Hill,40.876551,-73.91066,Blink Fitness,40.877271,-73.905595,Gym
8,Marble Hill,40.876551,-73.91066,Land & Sea Restaurant,40.877885,-73.905873,Seafood Restaurant
11,Marble Hill,40.876551,-73.91066,Starbucks,40.873755,-73.908613,Coffee Shop


In [57]:
filtered_manhattan_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Battery Park City,21,21,21,21,21,21
Carnegie Hill,39,39,39,39,39,39
Central Harlem,17,17,17,17,17,17
Chelsea,35,35,35,35,35,35
Chinatown,44,44,44,44,44,44
Civic Center,38,38,38,38,38,38
Clinton,45,45,45,45,45,45
East Harlem,14,14,14,14,14,14
East Village,44,44,44,44,44,44


In [58]:
print('There are {} uniques categories.'.format(len(filtered_manhattan_venues['Venue Category'].unique())))

There are 92 uniques categories.


**Analyze each neighborhood**

In [72]:
# one hot encoding
manhattan_onehot = pd.get_dummies(filtered_manhattan_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = filtered_manhattan_venues['Neighborhood'] 
# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]
manhattan_onehot.head()

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Boxing Gym,...,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
4,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
11,Marble Hill,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
#manhattan_grouped.shape
manhattan_grouped

,Neighborhood,Afghan Restaurant,African Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Asian Restaurant,Australian Restaurant,Austrian Restaurant,Boxing Gym,...,Tapas Restaurant,Thai Restaurant,Theme Restaurant,Turkish Restaurant,Udon Restaurant,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop
0,Battery Park City,0.000000,0.000000,0.047619,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.142857
1,Carnegie Hill,0.000000,0.000000,0.025641,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.025641,0.00000,0.025641,0.025641,0.076923
2,Central Harlem,0.000000,0.176471,0.117647,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.058824,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
3,Chelsea,0.000000,0.000000,0.085714,0.000000,0.000000,0.028571,0.000000,0.000000,0.000000,...,0.057143,0.028571,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.057143
4,Chinatown,0.000000,0.000000,0.090909,0.000000,0.000000,0.045455,0.000000,0.022727,0.000000,...,0.000000,0.022727,0.000000,0.000000,0.000000,0.022727,0.00000,0.068182,0.000000,0.000000
5,Civic Center,0.000000,0.000000,0.078947,0.000000,0.000000,0.026316,0.026316,0.000000,0.026316,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.052632,0.052632
6,Clinton,0.000000,0.000000,0.088889,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.022222,0.000000,0.000000,0.000000,0.022222,0.00000,0.000000,0.044444,0.066667
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.214286,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
8,East Village,0.000000,0.000000,0.045455,0.022727,0.022727,0.000000,0.000000,0.000000,0.000000,...,0.022727,0.000000,0.000000,0.000000,0.000000,0.068182,0.00000,0.045455,0.090909,0.022727
9,Financial District,0.000000,0.000000,0.100000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.040000


**Making the clusters to group similar neighborhoods"**

In [83]:
num_top_venues = 5

for hood in manhattan_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Battery Park City----
                venue  freq
0         Coffee Shop  0.24
1           Wine Shop  0.14
2                 Gym  0.14
3          Food Court  0.10
4  Italian Restaurant  0.10


----Carnegie Hill----
                  venue  freq
0           Coffee Shop  0.15
1             Wine Shop  0.08
2   Japanese Restaurant  0.08
3  Gym / Fitness Center  0.08
4                   Gym  0.08


----Central Harlem----
                 venue  freq
0   African Restaurant  0.18
1  American Restaurant  0.12
2   Seafood Restaurant  0.12
3    French Restaurant  0.12
4   Chinese Restaurant  0.12


----Chelsea----
                 venue  freq
0          Coffee Shop  0.20
1   Italian Restaurant  0.11
2  American Restaurant  0.09
3     Tapas Restaurant  0.06
4     Sushi Restaurant  0.06


----Chinatown----
                   venue  freq
0     Chinese Restaurant  0.18
1    American Restaurant  0.09
2  Vietnamese Restaurant  0.07
3      Hotpot Restaurant  0.07
4     Dim Sum Restaurant  0.05


---

In [76]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [77]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Coffee Shop,Wine Shop,Gym,Food Court,Italian Restaurant,Mexican Restaurant,Chinese Restaurant,Mediterranean Restaurant,Food Truck,Sushi Restaurant
1,Carnegie Hill,Coffee Shop,Wine Shop,Gym,French Restaurant,Japanese Restaurant,Gym / Fitness Center,Indian Restaurant,Italian Restaurant,Chinese Restaurant,Food Truck
2,Central Harlem,African Restaurant,French Restaurant,American Restaurant,Chinese Restaurant,Seafood Restaurant,Tapas Restaurant,Southern / Soul Food Restaurant,Gym,Gym / Fitness Center,Ethiopian Restaurant
3,Chelsea,Coffee Shop,Italian Restaurant,American Restaurant,Wine Shop,Seafood Restaurant,French Restaurant,Tapas Restaurant,Japanese Restaurant,Sushi Restaurant,Asian Restaurant
4,Chinatown,Chinese Restaurant,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Asian Restaurant,Malay Restaurant,Shanghai Restaurant


In [78]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 0, 2, 4, 0, 1, 2, 4])

In [79]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,4,American Restaurant,Coffee Shop,Gym,Seafood Restaurant,Wine Shop,Food Truck,Ethiopian Restaurant,Falafel Restaurant,Fast Food Restaurant,Filipino Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Chinese Restaurant,American Restaurant,Vietnamese Restaurant,Hotpot Restaurant,Coffee Shop,Dumpling Restaurant,Mexican Restaurant,Asian Restaurant,Malay Restaurant,Shanghai Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,2,Mexican Restaurant,Chinese Restaurant,Tapas Restaurant,Spanish Restaurant,Coffee Shop,New American Restaurant,Wine Shop,Latin American Restaurant,Gym,Metro Station
3,Manhattan,Inwood,40.867684,-73.921210,2,Mexican Restaurant,Restaurant,American Restaurant,Wine Bar,Chinese Restaurant,Wine Shop,Latin American Restaurant,Fast Food Restaurant,Coffee Shop,Seafood Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,2,Coffee Shop,Mexican Restaurant,Caribbean Restaurant,Indian Restaurant,Sushi Restaurant,Chinese Restaurant,Spanish Restaurant,Mediterranean Restaurant,Latin American Restaurant,Food Truck


In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

**Analize each of the Cluster**

In [81]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Manhattanville,Coffee Shop,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Food & Drink Shop,Climbing Gym,Spanish Restaurant,Sushi Restaurant,Cuban Restaurant,Ramen Restaurant
8,Upper East Side,Italian Restaurant,Coffee Shop,Gym / Fitness Center,French Restaurant,Mexican Restaurant,Sushi Restaurant,Wine Shop,American Restaurant,Latin American Restaurant,Chinese Restaurant
9,Yorkville,Italian Restaurant,Gym,Coffee Shop,Wine Shop,Sushi Restaurant,Mexican Restaurant,Japanese Restaurant,Vietnamese Restaurant,New American Restaurant,Gymnastics Gym
10,Lenox Hill,Italian Restaurant,Coffee Shop,Sushi Restaurant,Gym / Fitness Center,Gym,Wine Shop,Mexican Restaurant,Turkish Restaurant,Thai Restaurant,Japanese Restaurant
12,Upper West Side,Italian Restaurant,Wine Bar,Indian Restaurant,Coffee Shop,Mediterranean Restaurant,Restaurant,American Restaurant,Vegetarian / Vegan Restaurant,Middle Eastern Restaurant,Seafood Restaurant
13,Lincoln Square,Italian Restaurant,American Restaurant,Wine Shop,Gym / Fitness Center,French Restaurant,Gym,Coffee Shop,Mediterranean Restaurant,Food Truck,Wine Bar
14,Clinton,Gym / Fitness Center,Italian Restaurant,American Restaurant,Coffee Shop,Wine Shop,Gym,Wine Bar,Mediterranean Restaurant,New American Restaurant,Food Court
17,Chelsea,Coffee Shop,Italian Restaurant,American Restaurant,Wine Shop,Seafood Restaurant,French Restaurant,Tapas Restaurant,Japanese Restaurant,Sushi Restaurant,Asian Restaurant
18,Greenwich Village,Italian Restaurant,Sushi Restaurant,Gym,Indian Restaurant,French Restaurant,American Restaurant,Seafood Restaurant,Chinese Restaurant,Coffee Shop,Vietnamese Restaurant
21,Tribeca,American Restaurant,Italian Restaurant,Wine Bar,Coffee Shop,Greek Restaurant,Wine Shop,French Restaurant,Argentinian Restaurant,Korean Restaurant,New American Restaurant
